# DataLoader

In [ ]:
from PIL import Image
from torch.utils.data import Dataset
import glob

In [ ]:
# int(glob.glob('./datasets/images/*')[0].split('/')[-1].split('.')[0].split('_')[-1])

In [ ]:
img = Image.open(glob.glob('./datasets/images/*')[0])
img

In [ ]:
from torchvision import transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(64),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        # transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(64),
        transforms.CenterCrop(64),
        transforms.ToTensor(),
        # transforms.Normalize(mean, std)
    ]),
    'test': transforms.Compose([
        transforms.Resize(64),
        transforms.CenterCrop(64),
        transforms.ToTensor(),
        # transforms.Normalize(mean, std)
    ]),
}

In [ ]:
min(3,5)

In [ ]:
from PIL import Image
from torch.utils.data import Dataset
import glob

class span75speaker(Dataset):
    def __init__(self, image_path = './datasets/images', audio_path = './datasets/audios', transform=None, target_transform=None):
        self.images = glob.glob(f'{image_path}/*')  # Could be a list: ['./train/input/image_1.bmp', './train/input/image_2.bmp', ...]
        # self.audios = glob.glob(f'{audio_path}/*')  # Could be a nested list: [['./train/GT/image_1_1.bmp', './train/GT/image_1_2.bmp', ...], ['./train/GT/image_2_1.bmp', './train/GT/image_2_2.bmp', ...]]
        self.transform = transform
        self.target_transform = target_transform
        
    def __getitem__(self, index):
        image_name = self.images[index].split('/')[-1].split('.')[0].split('-')
        img = Image.open(self.images[index])
        if self.transform:
            img = self.transform(img)

        aud_embs = torch.load(f'./datasets/audios/{image_name[0]}.pt')
        aud_emb = aud_embs[:,min(int(image_name[-1]), aud_embs.size(1) - 1),:]

        return (img, aud_emb)

    def __len__(self):
        return len(self.images)

In [ ]:
traindata = span75speaker(transform = data_transforms['val'])
(img, aud) = next(iter(traindata))

In [ ]:
aud.shape

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img.permute(1, 2, 0))

# Data Pre-processing

### Preprocess Audio

In [ ]:
import librosa
import torch
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model

In [ ]:
# # Load model directly
# from transformers import AutoProcessor, AutoModelForCTC

# processor = AutoProcessor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
# modelphonene = AutoModelForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")

In [ ]:
modelphonene = model
modelphonene

In [ ]:
# import librosa
# import torch
# from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model

input_audio, sample_rate = librosa.load("/mnt/c/Users/PCM/Dropbox/span/sub006/2drt/audio/sub006_2drt_01_vcv1_r1_video.wav",  sr=16000)

model_name = "facebook/wav2vec2-lv-60-espeak-cv-ft"#"facebook/wav2vec2-base-960h"#"facebook/wav2vec2-large-xlsr-53"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
model = Wav2Vec2Model.from_pretrained(model_name)

i= feature_extractor(input_audio, return_tensors="pt", sampling_rate=sample_rate)
with torch.no_grad():
  o= model(i.input_values)

print(o.keys())
print(o.last_hidden_state.shape)
print(o.extract_features.shape)

In [ ]:
o.last_hidden_state.size(1)

In [ ]:
# aud_list = glob.glob(f'/mnt/c/Users/PCM/Dropbox/span/sub006/2drt/audio/*')[0]
# aud_list.split('/')[-1].split('.')[0]

In [ ]:
model_name = "facebook/wav2vec2-lv-60-espeak-cv-ft"#"facebook/wav2vec2-base-960h"#"facebook/wav2vec2-large-xlsr-53"
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
model = Wav2Vec2Model.from_pretrained(model_name)

def create_audio_emds(model, feature_extractor, in_path = '/mnt/c/Users/PCM/Dropbox/span/sub006/2drt/audio', out_path = './datasets/audios'):
    aud_list = glob.glob(f'{in_path}/*')
    for path in aud_list:
        name = path.split('/')[-1].split('.')[0]
        input_audio, sample_rate = librosa.load(f"{in_path}/{name}.wav",  sr=16000)

        i= feature_extractor(input_audio, return_tensors="pt", sampling_rate=sample_rate)
        with torch.no_grad():
            o = model(i.input_values)
        torch.save(o.last_hidden_state, f'{out_path}/{name}.pt')

# create_audio_emds(in_path = '/mnt/c/Users/PCM/Dropbox/span/sub006/2drt/audio', out_path = './datasets/audios/')

In [ ]:
# aaa = torch.load('/mnt/c/Users/PCM/Documents/GitHub/SPAN-rtmri/datasets/audios/sub001_2drt_01_vcv1_r2_video.pt')
# aaa.shape

In [ ]:
subjects = glob.glob('/mnt/c/Users/PCM/Dropbox/span/sub*')
for sub in subjects:
    print(sub)
    create_audio_emds(model, feature_extractor, in_path = f'{sub}/2drt/audio', out_path = './datasets/audios-eng-pho')

### Preprocess Video

In [ ]:
import glob
import subprocess

subjects = glob.glob('/mnt/c/Users/PCM/Dropbox/span/sub*')
for sub in subjects:
    vids = glob.glob(f'{sub}/2drt/video/*')
    # audio_foler = f'{sub}/2drt/audio'
    # subprocess.call(f'mkdir {audio_foler}', shell=True)
    for i in range(len(vids)):
        command = f"ffmpeg -i {vids[i]} -r 50 ./datasets/images/{vids[i].split('/')[-1].split('.')[0]}-%d.png"
        subprocess.call(command, shell=True)
        # !ffmpeg -i vids[i] -r 50 f'./datasets/images/{vids[i].split('/')[-1].split('.')[0]}-%d.png'

Sample video to frames in 50 fps

In [ ]:
!ffmpeg -i '/mnt/c/Users/PCM/Dropbox/span/sub006/2drt/video/sub006_2drt_01_vcv1_r1_video.mp4' -r 50 './datasets/images/sub006_2drt_01_vcv1_r1_video-%d.png'

# Modeling

In [ ]:
import torch
# from imagen_pytorch import Unet, Imagen
from imagen_pytorch import Unet, Imagen, ImagenTrainer
# from imagen_pytorch.data import Dataset
# unet for imagen

unet1 = Unet(
    dim = 32,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = 3,
    layer_attns = (False, True, True, True),
    layer_cross_attns = (False, True, True, True)
)

unet2 = Unet(
    dim = 32,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = (2, 4, 8, 8),
    layer_attns = (False, False, False, True),
    layer_cross_attns = (False, False, False, True)
)

# imagen, which contains the unets above (base unet and super resoluting ones)

imagen = Imagen(
    text_embed_dim=1024,
    unets = (unet1, unet2),
    image_sizes = (64, 64),
    timesteps = 1000,
    cond_drop_prob = 0.1
).cuda()

# mock images (get a lot of this) and text encodings from large T5

# text_embeds = o.extract_features.view(1413,1,512)[:10].cuda #torch.randn(1, 1, 512).cuda()
# images = torch.randn(1, 3, 64, 64).cuda()

dataset = span75speaker(transform = data_transforms['val'])

trainer = ImagenTrainer(
    imagen = imagen,
    split_valid_from_train = True # whether to split the validation dataset from the training
).cuda()
trainer.add_train_dataset(dataset, batch_size = 32)

for i in range(1,200000):
    loss = trainer.train_step(unet_number = 1, max_batch_size = 32)
    print(f'loss: {loss}')

    if not (i % 50):
        valid_loss = trainer.valid_step(unet_number = 1, max_batch_size = 32)
        print(f'valid loss: {valid_loss}')

    if not (i % 100) and trainer.is_main: # is_main makes sure this can run in distributed
        images = trainer.sample(text_embeds = aud.unsqueeze(0), batch_size = 1, return_pil_images = True, stop_at_unet_number=1) # returns List[Image]
        images[0].save(f'./sample_log/sample-{i // 100}.png')

# feed images into imagen, training each unet in the cascade

# for i in (1, 2):
#     loss = imagen(images, text_embeds = text_embeds, unet_number = i)
#     loss.backward()

# do the above for many many many many steps
# now you can sample an image based on the text embeddings from the cascading ddpm

# images = imagen.sample(texts = [
#     'a whale breaching from afar',
#     'young girl blowing out candles on her birthday cake',
#     'fireworks with blue and green sparkles'
# ], cond_scale = 3.)

# images.shape # (3, 3, 256, 256)

In [ ]:
torch.save(imagen.state_dict(), './imagen-span-070823')

In [ ]:
unet1 = Unet(
    dim = 32,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = 3,
    layer_attns = (False, True, True, True),
    layer_cross_attns = (False, True, True, True)
)

unet2 = Unet(
    dim = 32,
    cond_dim = 512,
    dim_mults = (1, 2, 4, 8),
    num_resnet_blocks = (2, 4, 8, 8),
    layer_attns = (False, False, False, True),
    layer_cross_attns = (False, False, False, True)
)

# imagen, which contains the unets above (base unet and super resoluting ones)

imagen = Imagen(
    text_embed_dim=1024,
    unets = (unet1, unet2),
    image_sizes = (64, 64),
    timesteps = 1000,
    cond_drop_prob = 0.1
).cuda()

imagen.load_state_dict(torch.load('./imagen-span-070823'))
imagen.eval()

In [ ]:

images = imagen.sample(text_embeds = torch.randn(1, 1, 512).cuda(), cond_scale = 3.)

images.shape # (3, 3, 256, 256)